# Combine echoes for each subject and session

Written by K. Garner, March 2021

Use this code first, to combine the multiecho images, using the weighted summation method from Pucket et al: https://www.sciencedirect.com/science/article/pii/S1053811917310194?via%3Dihub
Based on: https://github.com/Donders-Institute/multiecho/blob/master/multiecho/combination.py

Run manually on each sub and session

In [1]:
from nipype.interfaces.io import DataGrabber as dgrb
import nipype.pipeline as pe
import nipype as ni

from typing import List, Optional, Tuple
from pathlib import Path
import pandas as pd
import nibabel as nib
import numpy as np
import json

210303-12:27:18,994 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
# define session variables
sub = '05'
ses = '03'

In [3]:
dg = pe.Node(dgrb(infields=['sub', 'sess'], 
                            outfields=['func']), name='get-data')
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'func': [['sub', 'sess', 'sub']]}
dg.inputs.field_template = {'func': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/ses-%s/func/sub-%s_*-TR700_*_space-T1w_desc-preproc_bold.nii.gz'}

In [4]:
dg.inputs.sub = sub
dg.inputs.sess = ses
fs = dg.run()
fs.outputs

210303-12:27:19,290 nipype.workflow INFO:
	 [Node] Setting-up "get-data" in "/tmp/tmpll8pdeiy/get-data".
210303-12:27:19,297 nipype.workflow INFO:
	 [Node] Running "get-data" ("nipype.interfaces.io.DataGrabber")
210303-12:27:19,305 nipype.workflow INFO:
	 [Node] Finished "get-data".



func = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-05/ses-03/func/sub-05_ses-03_task-learnAtt_acq-TR700_echo-1_space-T1w_desc-preproc_bold.nii.gz', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-05/ses-03/func/sub-05_ses-03_task-learnAtt_acq-TR700_echo-2_space-T1w_desc-preproc_bold.nii.gz']

In [5]:
# get data and echoes in correct format for subsequent functions
datafiles = fs.outputs.func
TEs = [10, 30.56]

In [6]:
# def org_data(datafiles, TEs):
#     """Load all echoes and their TEs.
#     Return a list of tuples like:
#     [(echo1, TE1), (echo2, TE2), ..., (echoN, TEN)]
#     Here, echoN is a numpy array of loaded data.
#     """
#     s = np.argsort(TEs)
#     TEs = np.array(TEs)[s]
#     datafiles = np.array(datafiles)[s]
#     return[(nib.load(str(datafile)), TEs[n]) for n, datafile in enumerate(datafiles)], list(datafiles)

In [7]:
def org_data(datafiles, TEs):
    """Load all echoes and their TEs.
    Return a list of tuples like:
    [(echo1, TE1), (echo2, TE2), ..., (echoN, TEN)]
    Here, echoN is a numpy array of loaded data.
    """
    s = np.argsort(TEs)
    TEs = np.array(TEs)[s]
    datafiles = np.array(datafiles)[s]
    return[(nib.load(datafile), TEs[n]) for n, datafile in enumerate(datafiles)], list(datafiles)

In [8]:
def get_weights(echoes: List[Tuple[nib.Nifti1Image, float]], n_vols: int) -> np.array:
    """Compute weights from echoes described as a list of tuples,
    as sorted by org_data.
    w(tCNR) = TE * mean / sum(TE * mean) - adapted by K. Garner to reflect Puckett et al formula
    """
    def weight(echo: nib.Nifti1Image, TE: float):
        data = echo.get_fdata()
        mean = data[..., -n_vols:].mean(axis=-1)
        return TE * mean

    n_vols  = min(n_vols, echoes[0][0].shape[3])
    weights = [weight(echo, TE) for echo, TE in echoes]

#    weights = np.stack(weights, axis=-1)
#    mu = np.stack([weights.mean(axis=-1), weights.mean(axis=-1)], axis=-1)
#    return np.true_divide(weights, mu)
    return np.stack(weights, axis=-1) # because numpy average will normalise

In [9]:
def combine_them(datafiles: List[str],
                 TEs: List[int],
                 saveweights: bool = True) -> int:
    """General combine_them routine.
    Returns 1 if unmatching acquisitions
    Returns 0 upon success
    Currently supported algorithms:
    - Pucket et al 2018
    """

    # Load the data
    me_data, datafiles = org_data(datafiles, TEs)

    # Parse the output filenames
    datafile = datafiles[0]
    datastem = datafile.split(sep="_")
    outputname = '_'.join(datastem[0:4] + datastem[5:8])
    outputjsonstem = outputname.split(sep=".")
    outputjson = '.'.join([outputjsonstem[0] , 'json'])

    # truncate/match if incomplete runs 
    all_vols = [echo.shape[3] for echo, TE in me_data]
    if len(set(all_vols)) > 1:
        echos = np.stack([echo.get_fdata()[:, :, :, 0:min(all_vols)-1] for echo, TE in me_data], axis=-1)
        volumes = min(all_vols)
    else: 
        volumes = all_vols[0]
        echos = np.stack([echo.get_fdata() for echo, TE in me_data], axis=-1)

    # Compute the weights
    weights = get_weights(me_data, volumes)
    # Make the weights have the appropriate number of volumes.
    weights = np.tile(weights[:, :, :, np.newaxis, :], (1, 1, 1, echos.shape[3], 1))


    # Combine the images
    combined = np.average(echos, axis=-1, weights=weights)      # np.average normalizes the weights. No need to do that manually.
    affine   = me_data[0][0].affine
    header   = me_data[0][0].header
    combined = nib.Nifti1Image(np.nan_to_num(combined), affine, header)
    combined.to_filename(str(outputname))

    # Add a combined-echo json sidecar-file
    outputjson = outputjson
    data = {}
    data['EchoNumber'] = 1
    data['EchoTime'] = np.average([TE for echo, TE in me_data], weights=np.nanmean(weights[..., 0, :], axis=(0,1,2)))  # This seems to be the best we can do (the BIDS validator indicates there has to be a nr here, an empty value generates a warning)   
    with open(outputjson, 'w') as json_fid:
        json.dump(data, json_fid, indent=4)

    # Save the weights
    if saveweights: 
        fname         = datastem[0]+'_combined_weights.nii.gz'
        nifti_weights = nib.Nifti1Image(weights[..., 0, :], combined.affine, combined.header)
        nifti_weights.to_filename(str(fname))

    return 0

In [10]:
combine_them(datafiles, TEs)

0